# Resume of Labels

In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import yaml

## Resume by filename

In [ ]:
deep = "class_3"
files = os.listdir("data/"+ deep)
taxonomy_path = "taxonomy.yml"
print(len(files))

In [ ]:
with open(taxonomy_path) as taxpath:
    taxonomy = yaml.safe_load(taxpath)
    taxo = taxonomy[deep]

In [ ]:
df_dict = {"filename":[]}
for t in taxo.keys():
    df_dict[t] = []
for f in files:
    labels_f = torch.load(os.path.join("data",deep , f))
    sum_labels = np.sum(labels_f,axis=1)
    df_dict["filename"].append(f.split(".")[0])
    for i,t in enumerate(taxo.keys()):
        df_dict[t].append(int(sum_labels[i]))
df = pd.DataFrame(df_dict)
display(df)

In [ ]:
df.to_csv("resume_" + deep + ".csv",index = False)

## Resume by wetland

In [ ]:
def num2meta(num,df):
    df_n = df[(num>=df.Start) & (num<=df.End)]
    day,month,year = df_n.Date.values[0].split(' de ')
    wetland = df_n.Wetland.values[0]
    if month == 'enero':
        m = '01'
    elif month == 'abril':
        m = '04'
    elif month == 'julio':
        m = '07'
    elif month == 'octubre':
        m = '10'
    return day,m,year,wetland

In [ ]:
filenames = df.iloc[:,0].values
numfiles = [int(fn.replace("audio","")) for fn in filenames]
df_data = pd.read_csv("metadata.csv")

In [ ]:
day = []
month = []
year = []
wetland= []

for n in numfiles:
    d,m,y,w = num2meta(n,df_data)
    day.append(d)
    month.append(m)
    year.append(y)
    wetland.append(w)
df["day"] = day
df["month"] = month 
df ["year"] = year
df["wetland"] = wetland  

In [ ]:
df_wet = df.groupby(["wetland"]).sum()
df_date = df.groupby(["day","month","year"]).sum()

In [ ]:
df_wet

In [ ]:
df_date

In [ ]:
df_wet.to_csv("wet_resume_" + deep + ".csv",index   = True)
df_date.to_csv("date_resume_" + deep + ".csv",index = True)

In [ ]:
df_resume = df.drop(['filename','day','month','year','wetland'],axis=1)
df_resume = df_resume.sum()

In [ ]:
df_resume.to_csv('resume_'+deep+'.csv')